#  ICE_FLAVORS_RMV_VND_BRN_NAM_102623
   - module name: ICE_FLAVORS_RMV_VND_BRN_NAM_101623.ipnnb
   - purpose: Remove brand name from ECOMMERCE_DESCRIPTION
   - input: 
     - GTIN_NO and VND_BRN_NAM_102523.csv
     - GTIN_NO and VND_BRN_NAM_102523.csv
  
   - output:ICECREAM_BRN_NAM_MCH_102723.xlsx  
   - functions: 
   - writen by: Sophia Yue
   - date: 10.25.23 



In [2]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns= ['Sentence'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df['Drop'] = ''
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    for j in range(len(str_df)):
        str_1 = str_df.Sentence.loc[j]
        for i in range(len(str_df)-(j +1)):
            str_2 = str_df.Sentence.loc[i+j+1]
            if str_1 in str_2: 
                str_df.Drop.loc[j] = 'Yes'
                break
        if len(str_df) <2: 
            break
    return str_df[str_df.Drop != 'Yes'].Sentence.tolist()

In [3]:
#MAtch using 'token set ratio' with threshold
def flavor_match(item, flavor_list_org, cutoff = 75):
    #devowelized = [devowelize(i) for i in flavor_list_org]
    #derepeated = [remove_adjacent_repititions(i) for i in flavor_list_org]
    flavor_list = flavor_list_org #+ devowelized
    #flavor_list += derepeated
    #abbrev_maps = dict(zip(devowelized+ derepeated, flavor_list_org + flavor_list_org))
    #flavors_no_spaces = [i for i in flavor_list_org if not ' ' in i]
    #flavors_with_spaces = [i for i in Flavors_combo_bnj if ' ' in i]
    abbrev_maps = {}
    '''for i in flavors_no_spaces:
        for j in flavors_with_spaces:
            if j.replace(' ','') == i:
                abbrev_maps[i] = j'''
    abbrev_maps['NETFLIX']  = 'NETFLIX & CHILLED'   
    abbrev_maps['WAKE N BAKE'] = 'WAKE & NO BAKE'
    abbrev_maps['GIMMESMORE'] = "GIMME S'MORE!"
    abbrev_maps["GIMME S'MORE"] = "GIMME S'MORE!"
    abbrev_maps["COOKIES & CREAM"] = "COOKIES AND CREAM"
    abbrev_maps['MILK&COOKIES'] = 'MILK & COOKIES'
    # Add spaces to flavors as needed
    matched = process.extract(item, flavor_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process, limit = 7)
    flavor_shortlist = remove_substrings([i[0] for i in matched])
    matched = [i for i in matched if i[0] in flavor_shortlist]
    if len( flavor_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [abbrev_maps[i] if i in abbrev_maps.keys() else i for i in match_list]
        match_list = remove_substrings(match_list)
        # Second match using fuzz.partial_ratio
        rematched = process.extract(item, match_list,  scorer = fuzz.partial_ratio, processor=utils.default_process, limit = 5)
        re_max_score = rematched[0][1]
        final_rematch = [ i for i in rematched if i[1]== re_max_score]
        match_list =  [i[0] for i in final_rematch]
        '''
        match_list = [flavor_maps[i] if i in flavor_maps.keys() else i for i in match_list]
        if set(match_list) == {'STRAWBERRY','FRUIT'}: 
            match_list = ['FRUIT']#Drop Strawberry if fruit is the only other flavor
        if  set(match_list) != {'FRUIT'} : match_list = [ i for i in match_list if i != 'FRUIT']#Drop Fruit if it appears with something else
 
        if matched in multiflavor_dict.keys(): #Mapping multi-flavors to intended labels
            matched = multiflavor_dict[matched]
        '''
        flavor_count = len(set(match_list))
        matched = '|'.join(sorted(list(set(pd.Series(match_list)))))
        fuzz_score = max_score
    else: 
        matched = 'OTHER'
        flavor_count = 0
        fuzz_score = 0
    return (matched, flavor_count, fuzz_score)

In [4]:
import pandas as pd, numpy as np, re
from datetime import date
from rapidfuzz import process, fuzz,utils
pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\Kroger\\Data\\' 
path_atb = path + 'ATB\\'
path_atb_yue = path + 'ATB\\atb_yue\\'

In [5]:
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [6]:
replacements = {'MOCHI': '',
 ' VAN ': ' VANILLA ',
 ' CHOC ': ' CHOCOLATE ',
   'CHUNCKY' :'CHUNKY',    
   'MARSCAPONE':'MASCARPONE',             
 'NEOPOLITAN': 'NEAPOLITAN',
 ' BDAY ': ' BIRTHDAY ',
 ' CKE ': ' CAKE ',
 ' CRML ': ' CARAMEL ',
   'VNL ': ' VANILLA ', 
   ' VNLLA ': ' VANILLA ',            
 "'N'": 'AND',
 "'N ": 'AND ',
 " N' ": ' AND ',               
 ' & CREAM': ' AND CREAM',
 ' MNG ': ' MANGO ',
 ' STRWBRY ': ' STRAWBERRY ',
 ' STRWBRRY': ' STRAWBERRY ' ,
 ' STRAWBERRIES': 'STRAWBERRY' , 
 ' STRWB ': ' STRAWBERRY ' ,  
  ' STRW ': ' STRAWBERRY ',
  ' BLBRY ' : ' BLUEBERRY',
   ' CKIE DGH ': ' COOKIE DOUGH ',         
' FRUITY ': ' FRUIT '  ,
' FRUITS ': ' FRUIT '  ,             
 ' PNUT ': ' PEANUT ',
 ' BTTR ': ' BUTTER ',
 ' PNUT BTR ': ' PEANUT BUTTER ',              
 'CRM ': 'CREAM ',
 'CREME': 'CREAM',              
 'CHP': 'CHIP',
 'MNT': 'MINT',
 ' MINT CHIP': ' MINT CHOCOLATE CHIP',  
  ' COOKIE N CRM': ' COOKIES AND CREAM',
  ' CKIE CRM ': ' COOKIES AND CREAM ',
  'CKIES&CRM'   :'COOKIES AN CREAM',          
  ' PNTBTTR' :'PEANUT BUTTER',
   'PNUT BT' : 'PEANUT BUTTER',   
    ' ALM ' : ' ALMOND ',   
  'ICE CREAM':'',   
   'DOUBLE RAINBOW':'',           
  ',': ' ',
   '/': ' ',
  '>':' ',
  '<': ' '}


In [7]:
df_vnd_brn_name = f_read(path, 'GTIN_NO and VND_BRN_NAM_102523.csv')
df_vnd_brn_name['GTIN_NO'] = df_vnd_brn_name['GTIN_NO'].astype(np.int64)


file GTIN_NO and VND_BRN_NAM_102523.csv shape (5219536, 2) 
 columns: Index(['GTIN_NO', 'VND_BRN_NAM'], dtype='object')
dtypes: 
 GTIN_NO        object
VND_BRN_NAM    object
dtype: object
head: 
           GTIN_NO VND_BRN_NAM
0  00000000000000    Optifast
1  00000000000017   KENNICOTT
2  00000000000024         NaN
3  00000000000031         NaN
4  00000000000130         NaN


In [8]:
df_oth = f_read(path_atb, 'ICE_CREAM_FLAVORS_102523__with_scores.xlsx', sheet_name = 'OTHERs')
df_oth['VND_ECOM_DSC'] = df_oth.VND_ECOM_DSC.fillna('')
df_oth.shape

file ICE_CREAM_FLAVORS_102523__with_scores.xlsx shape (881, 7) 
 columns: Index(['GTIN_NO', 'ECOMMERCE_DESCRIPTION', 'VND_ECOM_DSC', 'TAG_DESCRIPTION',
       'PRODUCT_DESCRIPTION/MKT_MSG', 'Flavor', 'Match_Score'],
      dtype='object')
dtypes: 
 GTIN_NO                         int64
ECOMMERCE_DESCRIPTION          object
VND_ECOM_DSC                   object
TAG_DESCRIPTION                object
PRODUCT_DESCRIPTION/MKT_MSG    object
Flavor                         object
Match_Score                     int64
dtype: object
head: 
           GTIN_NO                       ECOMMERCE_DESCRIPTION  \
0     76840003891               Ben & Jerry's Ice Cream Pints   
1     76840100552  Ben & Jerry's® Cherry Garcia Frozen Yogurt   
2  10076840003898                                      (null)   
3  10076840100559                                      (null)   
4           22231                                      (null)   

                                   VND_ECOM_DSC            TAG_DESCRIPTIO

(881, 7)

In [9]:

df_oth = pd.merge(df_oth,  df_vnd_brn_name, on = 'GTIN_NO', how ='left' )
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN


In [10]:
df_oth["ECOM_DSC_MOD"] = df_oth.ECOMMERCE_DESCRIPTION.fillna('').str.upper().replace(replacements, regex = True)
df_oth.head()

,GTIN_NO,ECOMMERCE_DESCRIPTION,VND_ECOM_DSC,TAG_DESCRIPTION,PRODUCT_DESCRIPTION/MKT_MSG,Flavor,Match_Score,VND_BRN_NAM,ECOM_DSC_MOD
0,76840003891,Ben & Jerry's Ice Cream Pints,Ben & Jerry's Ice Cream Mixed 1 PT,BNJR FAV CARVE ICE CREAM,<p>A euphoric lineup of Ben &amp; Jerry's top ...,OTHER,0,Ben & Jerry's,BEN & JERRY'S PINTS
1,76840100552,Ben & Jerry's® Cherry Garcia Frozen Yogurt,Ben & Jerry's Ice Cream Frozen Yogurt 1 PT,BNJR CHRY GARCIA FRZ YGRT,<p>Ben &amp; Jerry's Cherry frozen yogurt with...,OTHER,0,Ben & Jerry's,BEN & JERRY'S® CHERRY GARCIA FROZEN YOGURT
2,10076840003898,(null),Ben & Jerry's Ice Cream Mixed 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
3,10076840100559,(null),Ben & Jerry's Ice Cream Frozen Yogurt 8 1 PT,(null),(null),OTHER,0,Ben & Jerry's,(NULL)
4,22231,(null),DELI GELATO SMALL,DELI GELATO SMALL,(null),OTHER,0,NaN,(NULL)


In [12]:
"""
 - columnwise comparison; not return match string;
 -  match string can get VND_BRN_NAM
 - can base on the score to determine to remove the bRN_NAME from  VND_ECOM_DSC
 - no data cleaning for VND_ECOM_DSC
 - the partial_ratio looks like  to get the best result
"""  
df_oth['ratio_score']    = df_oth.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['p_ratio_score']  = df_oth.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['ts_ratio_score'] = df_oth.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
df_oth['W_ratio_score']  = df_oth.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [13]:
"""
 - columnwise comparison; not return match string;
 - match string can get VND_BRN_NAM
 - can base on the score to determine to remove the bRN_NAME from  VND_ECOM_DSC
 - ECOM_DSC_MOD is the data from VND_ECOM_DSC with cleaning process
""" 
df_oth['ratio_score_m']    = df_oth.apply(lambda x:  fuzz.ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['p_ratio_score_m']  = df_oth.apply(lambda x:  fuzz.partial_ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['ts_ratio_score_m'] = df_oth.apply(lambda x:  fuzz.token_sort_ratio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1) 
df_oth['W_ratio_score_m']  = df_oth.apply(lambda x:  fuzz.WRatio(x.ECOM_DSC_MOD, x.VND_BRN_NAM), axis = 1)

In [14]:
vnd_map = list(set(df_oth.VND_BRN_NAM))
vnd_map[0:10]

['FLAVORICE',
 "N!ck's Ice Cream",
 "Friendly's",
 'AGLAMESIS BROTHERS',
 'ATA RETAIL',
 'Planet Oat',
 nan,
 'STO',
 'Joy',
 "Uncle Matt's"]

In [15]:
"""
  - apply vnd_map and flavor_match for lavor_match atching scoring
  - not columnwise comparison; The result could be misleading
"""

df_oth['vnd_mch'] = df_oth['ECOM_DSC_MOD'].apply(lambda x:flavor_match(x, flavor_list_org= vnd_map ))

C:\Users\iny2819\AppData\Local\Temp\ipykernel_13572\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2819\AppData\Local\Temp\ipykernel_13572\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2819\AppData\Local\Temp\ipykernel_13572\540123340.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_df.Drop.loc[j] = 'Yes'
C:\Users\iny2

In [16]:
# Process iriginal ice cream  file
ic_df = f_read(path_atb, 'ICECREAM_FLAVORS_101223_pipe_delim.csv', sep = '|') # shape (11086, 3) 
ic_df['VND_ECOM_DSC'].isnull().sum()
ic_df['VND_ECOM_DSC'] = ic_df.VND_ECOM_DSC.fillna('')

file ICECREAM_FLAVORS_101223_pipe_delim.csv shape (11086, 3) 
 columns: Index(['GTIN_NO', 'VND_ECOM_DSC', 'Flavor'], dtype='object')
dtypes: 
 GTIN_NO         object
VND_ECOM_DSC    object
Flavor          object
dtype: object
head: 
           GTIN_NO               VND_ECOM_DSC  Flavor
0  00000000013772  COFFEE BAR - BREWED COFFE  COFFEE
1  00000000022231          DELI GELATO SMALL   OTHER
2  00000000022248         DELI GELATO MEDIUM   OTHER
3  00000000022255          DELI GELATO LARGE   OTHER
4  00000000022262           DELI GELATO PINT   OTHER


In [17]:
ic_df['GTIN_NO'] =ic_df['GTIN_NO'].astype(np.int64)
ic_df = pd.merge(ic_df,  df_vnd_brn_name, on = 'GTIN_NO', how ='left' )

In [18]:
ic_df['ratio_score']    = ic_df.apply(lambda x:  fuzz.ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['p_ratio_score']  = ic_df.apply(lambda x:  fuzz.partial_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['ts_ratio_score'] = ic_df.apply(lambda x:  fuzz.token_sort_ratio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 
ic_df['W_ratio_score']  = ic_df.apply(lambda x:  fuzz.WRatio(x.VND_ECOM_DSC, x.VND_BRN_NAM), axis = 1) 

In [20]:
from datetime import date
from pandas import ExcelWriter
from pandas import ExcelFile

dte = date.today().strftime('%m%d%y')
excel_file = path_atb + "ICECREAM_BRN_NAM_MCH_" + dte + '.xlsx'
writer = pd.ExcelWriter(excel_file)
df_oth.to_excel(writer,'ICECREAM_OTH_BRN_NAM_MCH', index = False)
ic_df.to_excel(writer,'ICECREAM_ORG_BRN_NAM_MCH', index = False)
     
writer.close()

In [ ]:
stop

In [21]:
df_oth['VND_BRN_NAM'] = df_oth.VND_BRN_NAM.fillna('')

In [ ]:
# The following codes  not  working - KeyError: 'ECOM_DSC_MOD'
df_oth['vnd_mth1'] = df_oth.apply(lambda x:flavor_match(x['ECOM_DSC_MOD'], flavor_list_org= x['VND_BRN_NAM'],  cutoff = 75 ))

In [22]:
#NW TypeError: 'float' object is not iterable
ic_df['pe_ratio_score']     = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.ratio), axis = 1)
ic_df['pe_p_ratio_score']   = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.partial_ratio), axis = 1)
ic_df['pe_ts_ratio_score']  = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.token_sort_ratio), axis = 1)
ic_df['pe_W_ratio_score']   = ic_df.apply(lambda x: process.extract(x.VND_ECOM_DSC, x.VND_BRN_NAM, scorer= fuzz.WRatio), axis = 1)


TypeError: 'float' object is not iterable